In [1]:
import numpy as np
import itertools
import sympy as sp
import gurobipy as gp
from gurobipy import GRB

In [38]:
# Define the four functions
## Bin(INT) -> Bin(INT)
def identity(x):
    return x

def swap(x):
    return 1-x

def to0(_):
    return 0

def to1(_):
    return 1


basic_strategies = [identity, swap, to0, to1]
all_strategies = list(itertools.product(basic_strategies, repeat=2))

In [2]:
def generate_top(u):
    f_u, g_u = all_strategies[u]
    # print(f"f_{u} = {f_u.__name__} \ng_{u} = {g_u.__name__}\n")

    # value 1 at point(a,b) = (f_u(x),g_u(y)), zero everywhere else
    def middle_entry(x,y):
        # value 1 at (f_u(x),g_u(y)) zero everywhere else
        middle = np.zeros((2,2), dtype=int)
        middle[f_u(x), g_u(y)] = 1
        return middle

    top_level = np.array([
                [middle_entry(0,0), middle_entry(1,0)], 
                [middle_entry(0,1), middle_entry(1,1)]
                ]) 

    return top_level

# generate_top(15)

In [72]:
# generate_top(4)

In [3]:
# declaring Gurobi variable W_i w/ i in {0,1,...,15}, and declaring v
m = gp.Model()
W = m.addVars(16, lb=0, ub=1, vtype=GRB.CONTINUOUS, name="W")
m.update()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-09


In [4]:
# print("feasible distribution?", sum(dist_ABXY.values()) == 1)

# # # get P_AB|XY from P_ABXY
# def P_AB_giv_XY(A,B, X, Y):
#     P_ABXY = sum([dist_ABXY[(a,b,x,y)] for a,b,x,y in dist_ABXY if a==A and b==B and x==X and y==Y])
#     P_AB = sum([dist_ABXY[(a,b,x,y)] for a,b,x,y in dist_ABXY if a==A and b==B])
#     return P_ABXY/P_AB


# P_AB_giv_XY = ...

In [6]:
internal_gurobi_expr = sum([W[i]*generate_top(i) for i in range(16)])


internal_gurobi_expr $[x,y,a,b]$  
P_AB_giv_XY $(a,b,x,y)$

Found summed strategies matrices for all U! (internal_gurobi_exp)

## Gurobi Constraints

In [7]:
# Add the constraint to the model
m.addConstr(W.sum() == 1, "sum(W_i) == 1")
for x,y, a,b in itertools.product(range(2), repeat=4):
    m.addConstr(internal_gurobi_expr[x,y,a,b] == P_AB_giv_XY(a,b,x,y), f"P({a},{b}|{x},{y}) == {internal_gurobi_expr[x,y,a,b]} == {P_AB_giv_XY(a,b,x,y)}")

# for cons in m.getConstrs():
#     print(cons)

# reset gurobi constraints
# m.remove(m.getConstrs())

m.update()

In [8]:
m.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-11800H @ 2.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 17 rows, 16 columns and 80 nonzeros
Model fingerprint: 0xd735e9eb
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e-02, 1e+00]
Presolve removed 3 rows and 10 columns
Presolve time: 0.00s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Infeasible model
